# Sayou Wrapper Quick Start

**Sayou Fabric**의 지식 포장 단계인 `sayou-wrapper`를 시연합니다.
이전 단계(`sayou-chunking`)에서 잘린 청크들을 입력받아, 사유존 표준 스키마인 **SayouNode**로 변환합니다. 이 과정에서 단순 텍스트는 **Ontology Class**(`Topic`, `Table` 등)를 부여받고, 문맥은 **Predicate**(`hasParent`)로 연결됩니다.

In [ ]:
import json
import logging
from sayou.wrapper.pipeline import WrapperPipeline

# 로그 설정
logging.basicConfig(level=logging.INFO, format='%(message)s')

print("Import Successful!")

## 1. 파이프라인 초기화

Wrapper 파이프라인을 생성합니다. 기본적으로 `DocumentChunkAdapter`가 로드됩니다.

In [ ]:
pipeline = WrapperPipeline()
pipeline.initialize()

## 2. 데이터 준비 (Simulation)

`sayou-chunking`의 결과물(JSON 파일)이 있다고 가정하고 로드하거나, 테스트용 더미 데이터를 생성합니다.

In [ ]:
# 시나리오: Chunking 결과물 (Header 1개 + Child Text 2개)
chunks_data = [
    {
        "content": "# Introduction to AI",
        "metadata": {
            "chunk_id": "doc1_h_0",
            "semantic_type": "h1",
            "is_header": True,
            "source": "ai_intro.md",
            "page_num": 1
        }
    },
    {
        "content": "AI stands for Artificial Intelligence.",
        "metadata": {
            "chunk_id": "doc1_p_0",
            "semantic_type": "text",
            "parent_id": "doc1_h_0",  # 부모(Header)를 가리킴
            "source": "ai_intro.md",
            "page_num": 1
        }
    },
    {
        "content": "| Model | Type |",
        "metadata": {
            "chunk_id": "doc1_t_0",
            "semantic_type": "table",
            "parent_id": "doc1_h_0",
            "source": "ai_intro.md"
        }
    }
]

# (옵션) 실제 파일에서 로드하는 경우
# input_file = "chunks_output.json"
# if os.path.exists(input_file):
#     with open(input_file, "r", encoding="utf-8") as f:
#         chunks_data = json.load(f)

print(f"Ready to process {len(chunks_data)} chunks.")

## 3. 실행 (Run Wrapper)

`document_chunk` 전략을 사용하여 변환을 수행합니다.
* **Text** -> `sayou:TextFragment`
* **Header** -> `sayou:Topic`
* **Table** -> `sayou:Table`
* **Relationship** -> `sayou:hasParent` 연결

In [ ]:
try:
    # 실행
    output = pipeline.run(chunks_data, strategy="document_chunk")
    
    print(f"✅ Generated {len(output.nodes)} SayouNodes.\n")
    
    # 결과 검증
    for node in output.nodes:
        print(f"🔹 [{node.node_class}] {node.node_id}")
        print(f"   Name: {node.friendly_name}")
        
        # 속성 출력
        if "schema:text" in node.attributes:
            print(f"   Text: {node.attributes['schema:text'][:30]}...")
        
        # 관계 출력
        if node.relationships:
            print(f"   Rels: {node.relationships}")
        print("-" * 40)

except Exception as e:
    print(f"❌ Error: {e}")

## 4. 최종 산출물 저장

이 데이터는 이제 `sayou-assembler`나 `sayou-loader`로 넘어갈 준비가 되었습니다.

In [ ]:
# Pydantic Model -> JSON String
final_json = output.model_dump_json(indent=2)

with open("wrapper_output.json", "w", encoding="utf-8") as f:
    f.write(final_json)
    
print("Saved to wrapper_output.json")